In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-bbf1  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 10:05 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'body0901' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
postings_gcp/0_posting_locs.pickle
postings_gcp/100_posting_locs.pickle
postings_gcp/101_posting_locs.pickle
postings_gcp/102_posting_locs.pickle
postings_gcp/103_posting_locs.pickle
postings_gcp/104_posting_locs.pickle
postings_gcp/105_posting_locs.pickle
postings_gcp/106_posting_locs.pickle
postings_gcp/107_posting_locs.pickle
postings_gcp/108_posting_locs.pickle
postings_gcp/109_posting_locs.pickle
postings_gcp/10_posting_locs.pickle
postings_gcp/110_posting_locs.pickle
postings_gcp/111_posting_locs.pickle
postings_gcp/112_posting_locs.pickle
postings_gcp/113_posting_locs.pickle
postings_gcp/114_posting_locs.pickle
postings_gcp/115_posting_locs.pickle
postings_gcp/116_posting_locs.pickle
postings_gcp/117_posting_locs.pickle
postings_gcp/118_posting_locs.pickle
postings_gcp/119_posting_locs.pickle
postings_gcp/11_posting_locs.pickle
postings_gcp/120_posting_locs.pickle
postings_gcp/121_posting_locs.pickle
postings_gcp/122_posting_locs.pickle
postings_gcp/123_posting_lo

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

In [ ]:
import json

with open('/home/dataproc/queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:

# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_body_gcp.py

inverted_index_body_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_body_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_body_gcp import InvertedIndex

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  tokens = [x for x in tokens if x not in all_stopwords]
  pairs_tokens = {}
  c = Counter(tokens)
  for token in tokens:
    tf = c[token]
    pairs_tokens[token] = (id, pairs[id])
  pairs = list(pairs_tokens.items())
  return pairs


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  sorted_pl = sorted(unsorted_pl)
  return sorted_pl


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  token_df_pairs = postings.map(lambda item: (item[0], len(item[1])))
  return token_df_pairs


def partition_postings_and_write(postings, indexNum):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  token_loc_pairs = postings.map(lambda item: (token2bucket_id(item[0]), item))
  buckets = token_loc_pairs.groupByKey().mapValues(list)
  return buckets.map(lambda bucket: InvertedIndex.write_a_posting_list((bucket[0], bucket[1]),bucket_name, indexNum))

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts_body = doc_text_pairs.flatMap(lambda x: word_count(x[1], x[0]))
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_body_filtered = postings_body.filter(lambda x: len(x[1])>10)
del postings_body
w2df_body = calculate_df(postings_body_filtered)
w2df_body_dict = w2df_body.collectAsMap()
del w2df_body
# partition posting lists and write out
posting_locs_list = partition_postings_and_write(postings_body_filtered, 200).collect()
index_const_time = time() - t_start

In [ ]:
 test index construction time
# assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    

In [ ]:
def create_DL_dict(item):
  tokens = [token.group() for token in RE_WORD.finditer(item[1].lower())] 
  tokens = [x for x in tokens if x not in all_stopwords]
  try:
    elem = [(item[0], DL_body_arr[DL_body_arr.index(item[0])] + (len(tokens)))]
  except:
    elem = [((item[0], 0 + (len(tokens))))]
  return elem
    
#calculate DL for body
temp_DL_dict = doc_text_pairs.flatMap(lambda item: create_DL_dict(item))
DL_body_dict = temp_DL_dict.collectAsMap()
del temp_DL_dict


In [ ]:
#title dict for body and title
def create_title_dict(item):
  return [(item[0], item[1])]

doc_title_pairs = parquetFile.select("id", "title").rdd

#title dict for body and title
temp_title_dict = doc_title_pairs.flatMap(lambda item: create_title_dict(item))
title_dict = temp_title_dict.collectAsMap()
del temp_title_dict

In [ ]:
!gsutil ls -lh $index_dst

gs://body0901/
gs://dataproc-staging-us-central1-344044549712-us432d09/
gs://dataproc-temp-us-central1-344044549712-vzn419ht/


In [ ]:
# Create inverted index instance
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_body_dict
# Add the DL dictionary to the inverted index
inverted_body.DL = DL_body_dict
# Add the title dictionary to the inverted index
inverted_body.titles = title_dict
# Add the pr dictionary to the inverted index
inverted_body.pr = {}
# write the global stats out
inverted_body.write_index('.', 'index_body')
# upload to gs
index_src = "index_body.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst